In [2]:
#Exercise 1: Project 4 data cleaning
# 1. Read the excel file using the pandas library
# 2. Apply at least three data cleaning techniques of which you think are suitable for the data set. You can choose from the ones above, or think of a technique yourself.
# 3. Structure your code by combining all functionalities into a function. This function should return the cleaned data set
import pandas as pd
data_pj = pd.read_excel('dataProject4.xlsx') 

print(data_pj.shape)
data_pj.head()

(0, 0)


""


In [5]:
#You have been given a file about hotel bookings, you can find these data in the file
#‘hotelBookings.xlsx’. These data contain the number of bookings of the hotel ‘Resort Hotel’,
#which is spread over different countries. This file contains a lot of errors, and needs to be
#cleaned. For this exercise, you need to read the Excel file, analyse the data, find as many
#mistakes as possible and correct them. In particular:
#1. Read the excel file using the pandas library
#2. Inspect the data and find as many mistakes you can.
#3. Clean the data, which decisions did you make to solve the errors?

df_hotel=pd.read_excel('hotelBookings.xlsx')
print(df_hotel.shape)
df_hotel.head(5)

(1000, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0.0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0.0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1.0,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1.0,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2.0,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [6]:
#dropping duplicates (keeping the first entry) and checking for missing values
if df_hotel.duplicated().any():
    print("Warning: The data contains duplicate rows.")
    # Remove duplicate rows
    print(df_hotel.duplicated().sum())
    df_hotel=df_hotel.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found")

if df_hotel.isnull().values.any():
    print("Warning: The data contains missing values.")
else:
    print("No missing values found")

#should we print the duplicates first to inspect?
#we will handle missing data later
print(df_hotel.shape)

57
Duplicates removed.
(943, 32)


In [8]:
#seeing where we have missing values 
print(df_hotel.isnull().sum()) 

#arrival date month - dropping all 14, since check in date is important and its hard to calculate beased on week number (maybe someday ill do this too)
#country - one missing, dropping
#agent - not dropping, there's no way to calculate an 'average', but it is also not as important as other data imo; also by dropping like 100 rows we would lose other valuable data
#company - basically 99% missing, but not dropping any rows because of it
df_hotel=df_hotel.drop(df_hotel[df_hotel['arrival_date_month'].isnull()].index)
df_hotel=df_hotel.drop(df_hotel[df_hotel['country'].isnull()].index)
print(df_hotel.shape)

#i ran this code multiple times so you cant really see which ones were missing in the beginning but at least now we can check that their count is 0

hotel                               0
is_canceled                         0
lead_time                           0
arrival_date_year                   0
arrival_date_month                  0
arrival_date_week_number            0
arrival_date_day_of_month           0
stays_in_weekend_nights             0
stays_in_week_nights                0
adults                              0
children                            0
babies                              0
meal                                0
country                             0
market_segment                      0
distribution_channel                0
is_repeated_guest                   0
previous_cancellations              0
previous_bookings_not_canceled      0
reserved_room_type                  0
assigned_room_type                  0
booking_changes                     0
deposit_type                        0
agent                              90
company                           917
days_in_waiting_list                0
customer_typ

In [9]:
#filtering out extreme values
#print(df_hotel['adults'].unique())
column_list = df_hotel.columns.values.tolist()
for column_name in column_list:
    if column_name != 'reservation_status_date' and column_name != 'adr' : #that list was too long
        print(column_name, df_hotel[column_name].unique())

#did you know that the mom of most children had 69 babies?? im dropping any entry with over 70 children
df_hotel=df_hotel.drop(df_hotel[df_hotel['adults'] > 70].index | df_hotel[df_hotel['children'] > 70].index)
#2099 is the year when im gonna finish this assignment
df_hotel=df_hotel.drop(df_hotel[df_hotel['arrival_date_year'] == 2099].index)
#theres people that spent a non integer number of nights at the hotel and if that is true they should be billed accordingly
df_hotel=df_hotel.drop(df_hotel[df_hotel['stays_in_week_nights'] == 4.3].index) #i couldnt find a function that sorts by data type (keep only integers)
#country numbers 2 and 3 (because romania is always number 1)
df_hotel=df_hotel.drop(df_hotel[df_hotel['country'] == 2].index | df_hotel[df_hotel['country'] == 3].index)

#i dont know what the units of measurement is for lead time, adr (what even is that), agent or company is so i cant tell if they are wrong data entries or outliers or whatnot


hotel ['Resort Hotel']
is_canceled [0 1]
lead_time [342 737   7  13  14   0   9  85  75  23  35  68  18  37  12  72 127  78
  48  60  77  99  95  96  69  45  40  15  36  43  70  16 107  47 113  90
  50  93  76   3   1  10   5  17  51  71  63  62 101   2  81 368 364 324
  79  21 109 102   4  98  92  26 115  86  52  29  30  73  33  32   8 100
  44  80  97  64  39  34  27  82  94 110 111  84  66 104  28 258 112  65
  67  55  88  54 292  83 105 280 394  24 103 366 249  22  91  11 108 106
  31  87  41 304 117  59  53 116  42  58 321  38 118  56  49 317   6  57
  19  25 315 123  46  89  61 312 299 130  74 298 119  20 286 136 124 327
 131 460 140 114 139 129 122 137 126]
arrival_date_year [2015 2099]
arrival_date_month ['July' 'August']
arrival_date_week_number [27 28 29 30 31 32 33]
arrival_date_day_of_month [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
stays_in_weekend_nights [0 1 2 4 3 6]
stays_in_week_nights [ 0.   1.   2.   3.   4.   4.3 

ValueError: cannot reshape array of size 2 into shape (1,)

In [10]:
#adding some interesting information
df_hotel['nights_in_hotel'] = df_hotel['stays_in_weekend_nights'] + df_hotel['stays_in_week_nights']
df_hotel['number_guests'] = df_hotel['adults'] + df_hotel['children']+df_hotel['babies']

df_hotel=df_hotel.drop(df_hotel[df_hotel['number_guests'] == 0].index) #these are useless entries
df_hotel=df_hotel.drop(df_hotel[(df_hotel['nights_in_hotel']==0) & (df_hotel['reservation_status']=='Check-Out')].index)

df_hotel.shape

(906, 34)

In [11]:
from datetime import datetime
df_hotel['reservation_status_date'] = pd.to_datetime(df_hotel['reservation_status_date'])
df_hotel['reservation_status_european_date'] = df_hotel['reservation_status_date'].dt.strftime('%d-%m-%Y') #because we respect ourselves


In [13]:
df_hotel['arrival_date_month'].replace('July', '07', inplace=True) 
df_hotel['arrival_date_month'].replace('August', '08', inplace=True)
df_hotel.head()

df_hotel['arrival_date'] = df_hotel['arrival_date_day_of_month'].astype(str)+'-'+df_hotel['arrival_date_month'].astype(str)+'-'+df_hotel['arrival_date_year'].astype(str)
df_hotel['arrival_date'] = pd.to_datetime(df_hotel['arrival_date'])
df_hotel.head()

ValueError: time data "13-07-2015" doesn't match format "%m-%d-%Y", at position 12. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [14]:
df_hotel.replace('null', 'NULL', inplace=True)


In [15]:
df_hotel.loc[df_hotel['country'] == '  ROU', 'country'] = 'ROU' #romanians are always causing troubles
df_hotel.loc[df_hotel['meal'] == '    HB', 'meal'] = 'HB'
df_hotel.loc[df_hotel['meal'] == '             BB', 'meal'] = 'BB'
df_hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,nights_in_hotel,number_guests,reservation_status_european_date,arrival_date
2,Resort Hotel,0,7,2015,07,27,1,0,1.0,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,1.0,1,02-07-2015,1-07-2015
3,Resort Hotel,0,13,2015,07,27,1,0,1.0,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,1.0,1,02-07-2015,1-07-2015
4,Resort Hotel,0,14,2015,07,27,1,0,2.0,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,2.0,2,03-07-2015,1-07-2015
6,Resort Hotel,0,0,2015,07,27,1,0,2.0,2,...,Transient,107.0,0,0,Check-Out,2015-07-03,2.0,2,03-07-2015,1-07-2015
7,Resort Hotel,0,9,2015,07,27,1,0,2.0,2,...,Transient,103.0,0,1,Check-Out,2015-07-03,2.0,2,03-07-2015,1-07-2015
